In [1]:
import os

print(os.path.abspath(os.curdir))

d:\Data Kuliah\Semester 9\DFRWS EU 26\droptc-dev\notebooks


In [2]:
import os
os.chdir("..")

In [3]:
print(os.path.abspath(os.curdir))

d:\Data Kuliah\Semester 9\DFRWS EU 26\droptc-dev


### Sentece-level results

In [4]:
import json
import pandas as pd
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score, matthews_corrcoef

raw2pro = {
    'normal': 'Normal',
    'HwFlt': 'HardwareFault',
    'Swflt': 'SoftwareFault',
    'SurEnv': 'SurroundingEnvironment',
    'ConfIss': 'ParamViolation',
    'CommIss': "CommunicationIssue",
    'VioReg': 'RegulationViolation'
}

analysis_level = 'sentence'
scenarios = ['droptc', 'drolove', 'dronelog', 'neurallog', 'transsentlog']
loss_functions = ['ce', 'focal']
class_weights = ['uniform', 'inverse', 'balanced']
recap_overall = pd.DataFrame()
recap_per_class = pd.DataFrame()
train_df = pd.read_excel(os.path.join('dataset', 'train_sentence.xlsx'))
# true_df = pd.read_excel(os.path.join('dataset', 'test_sentence.xlsx'))
train_sentences = set(train_df['sentence'].unique())
for scenario_name in scenarios:
    source_dir = os.path.join('experiments', scenario_name, analysis_level)
    embeddings = os.listdir(source_dir)
    for embedding in embeddings:
        embedding_path = os.path.join(source_dir, embedding)
        freezes = os.listdir(embedding_path)
        for freeze in freezes:
            freeze_path = os.path.join(embedding_path, freeze)
            loss_weights = os.listdir(freeze_path)
            for loss_weight in loss_weights:
                loss_fc, class_weight = loss_weight.split('-')
                loss_weight_path = os.path.join(freeze_path, loss_weight)
                seeds = os.listdir(loss_weight_path)
                for seed in seeds:
                    seed_path = os.path.join(loss_weight_path, str(seed))
                    all_files = os.listdir(seed_path)
                    eval_report_path = os.path.join(seed_path, 'evaluation_report.json')
                    scenario_path = os.path.join(seed_path, 'scenario_arguments.json')
                    with open(eval_report_path) as eval_report_file:
                        eval_report = json.load(eval_report_file)
                        with open(scenario_path) as scenario_json_file:
                            scenario = json.load(scenario_json_file)
                            prediction_path = os.path.join(seed_path, 'prediction.xlsx')
                            prediction_df = pd.read_excel(prediction_path)

                            # Identify non-leaked samples
                            non_leaked_mask = ~prediction_df['sentence'].isin(train_sentences)

                            # Filter prediction and ground truth
                            filtered_pred_df = prediction_df[non_leaked_mask]
                            # save the filtered_pred into excel file for validation
                            filtered_pred_df.to_excel(os.path.join(seed_path, 'prediction_filtered.xlsx'))

                            # Compute evaluation
                            # Extract labels
                            y_true = filtered_pred_df['label'].values
                            y_pred = filtered_pred_df['pred'].values
                            unique_acc = round(accuracy_score(y_true, y_pred), 3)
                            unique_precision = round(precision_score(y_true, y_pred, average='weighted', zero_division=0), 3)
                            unique_recall = round(recall_score(y_true, y_pred, average='weighted', zero_division=0), 3)
                            unique_f1 = round(f1_score(y_true, y_pred, average='weighted', zero_division=0), 3)
                            unique_mcc = matthews_corrcoef(y_true, y_pred)
                            mcc = matthews_corrcoef(prediction_df['label'].to_list(), prediction_df['pred'].to_list())
                            

                            confidence_avg = prediction_df['prob'].mean()
                            confidence_std = prediction_df['prob'].std()
                            accuracy = float(round(eval_report['accuracy'], 3))
                            precision = float(round(eval_report['weighted_avg']['precision'], 3))
                            recall = float(round(eval_report['weighted_avg']['recall'], 3))
                            f1score = float(round(eval_report['weighted_avg']['f1-score'], 3))
                            best_epoch = int(scenario['best_epoch'])
                            columns = ['dataset', 'scenario', 'embedding', 'freeze_param', 'loss_fc', 'class_weight', 'seed', 'best_epoch', 'accuracy', 'precision', 'recall', 'f1_score', 'mcc', 'sample', 'unq_acc', 'unq_pre', 'unq_rec', 'unq_f1', 'unq_mcc', 'unq_sample', 'confidence_avg', 'confidence_std']
                            row_list = ['sentence', scenario_name, embedding, freeze, loss_fc, class_weight, seed, best_epoch, accuracy, precision, recall, f1score, mcc, len(prediction_df), unique_acc, unique_precision, unique_recall, unique_f1, unique_mcc, len(filtered_pred_df), confidence_avg, confidence_std]
                            scenario_df = pd.DataFrame([row_list], index=None, columns=columns)
                            recap_overall = pd.concat([recap_overall, scenario_df], ignore_index=True)
# if not os.path.exists(os.path.join('experiments', 'recap', 'sentence')):
os.makedirs(os.path.join('experiments', 'recap', 'sentence'), exist_ok=True)
recap_overall.to_excel(os.path.join(os.path.join('experiments', 'recap', 'sentence'), f'recap_overall_sentence_new.xlsx'), index=False, float_format='%.3f')

In [ ]:
# recap_overall = pd.read_excel(os.path.join(os.path.join('recap', 'sentence'), f'recap_overall_sentence.xlsx'))

In [5]:
grouper = ['dataset', 'scenario', 'embedding', 'loss_fc', 'class_weight', 'freeze_param']
selected_columns = ['accuracy', 'precision', 'recall', 'f1_score', 'mcc', 'unq_acc', 'unq_pre', 'unq_rec', 'unq_f1', 'unq_mcc']

multitask_agg = recap_overall.groupby(grouper)[selected_columns].agg(['mean', 'std']).sort_values(('f1_score', 'mean'), ascending=True)
multitask_agg.to_excel(os.path.join('experiments', 'recap', f'recap_overall_sentence_f1_score.xlsx'), index=True, float_format='%.3f')
multitask_agg

accuracy  \
                                                                                              mean   
dataset  scenario     embedding                         loss_fc class_weight freeze_param            
sentence droptc       DroPTC-all-mpnet-base-v2-sentence focal   balanced     freeze         0.3752   
                      all-mpnet-base-v2                 ce      uniform      freeze         0.7139   
                      all-MiniLM-L6-v2                  ce      uniform      freeze         0.7167   
                      bert-base-uncased                 ce      uniform      freeze         0.7355   
                      modern-bert                       ce      uniform      freeze         0.7401   
                      neo-bert                          ce      uniform      freeze         0.8560   
                      DroPTC-all-mpnet-base-v2-sentence ce      uniformold   freeze         0.9135   
                                                                uniform      freeze         0.9135   
                      modern-bert                       ce      uniform      unfreeze       0.9075   
         dronelog     DroPTC-all-mpnet-base-v2-sentence ce      uniform      unfreeze       0.9111   
         droptc       neo-bert                          ce      uniform      unfreeze       0.9134   
                      DroPTC-all-MiniLM-L6-v2-sentence  focal   balanced     unfreeze       0.9124   
         drolove      bert-base-uncased                 ce      uniform      unfreeze       0.9149   
         droptc       bert-base-uncased                 ce      uniform      unfreeze       0.9177   
         neurallog    bert-base-uncased                 ce      uniform      unfreeze       0.9168   
         droptc       DroPTC-all-MiniLM-L6-v2-sentence  focal   inverse      unfreeze       0.9174   
         transsentlog bert-base-uncased                 ce      uniform      unfreeze       0.9186   
         droptc       DroPTC-all-MiniLM-L6-v2-sentence  ce      balanced     unfreeze       0.9212   
                                                                inverse      unfreeze       0.9212   
                                                        focal   uniform      unfreeze       0.9224   
                                                        ce      uniform      unfreeze       0.9218   
                      DroPTC-all-mpnet-base-v2-sentence focal   uniform      freeze         0.9247   
                      all-MiniLM-L6-v2                  ce      uniform      unfreeze       0.9214   
                      all-mpnet-base-v2                 ce      inverse      unfreeze       0.9233   
                      DroPTC-all-MiniLM-L6-v2-sentence  ce      balanced     freeze         0.9230   
                                                        focal   uniform      freeze         0.9230   
                                                        ce      inverse      freeze         0.9230   
                                                                uniform      freeze         0.9230   
                                                                uniformold   freeze         0.9230   
                                                        focal   balanced     freeze         0.9227   
                                                                inverse      freeze         0.9235   
                                                        ce      uniformold   unfreeze       0.9238   
                      all-mpnet-base-v2                 ce      balanced     unfreeze       0.9244   
                      DroPTC-all-mpnet-base-v2-sentence focal   balanced     unfreeze       0.9253   
                                                        ce      inverse      freeze         0.9265   
                                                                balanced     freeze         0.9265   
                                                                uniform      unfreeze       0.9275   
          

In [7]:
grouper = ['dataset', 'scenario', 'embedding', 'loss_fc', 'class_weight', 'freeze_param']
selected_columns = ['accuracy', 'precision', 'recall', 'f1_score', 'mcc', 'unq_acc', 'unq_pre', 'unq_rec', 'unq_f1', 'unq_mcc']

multitask_agg = recap_overall.groupby(grouper)[selected_columns].agg(['mean', 'std'])
multitask_agg.to_excel(os.path.join('experiments', 'recap', f'recap_overall_sentence_f1_score_unsorted.xlsx'), index=True, float_format='%.3f')
multitask_agg

accuracy  \
                                                                                              mean   
dataset  scenario     embedding                         loss_fc class_weight freeze_param            
sentence drolove      bert-base-uncased                 ce      uniform      unfreeze       0.9149   
         dronelog     DroPTC-all-mpnet-base-v2-sentence ce      uniform      freeze         0.9280   
                                                                             unfreeze       0.9111   
         droptc       DroPTC-all-MiniLM-L6-v2-sentence  ce      balanced     freeze         0.9230   
                                                                             unfreeze       0.9212   
                                                                inverse      freeze         0.9230   
                                                                             unfreeze       0.9212   
                                                                uniform      freeze         0.9230   
                                                                             unfreeze       0.9218   
                                                                uniformold   freeze         0.9230   
                                                                             unfreeze       0.9238   
                                                        focal   balanced     freeze         0.9227   
                                                                             unfreeze       0.9124   
                                                                inverse      freeze         0.9235   
                                                                             unfreeze       0.9174   
                                                                uniform      freeze         0.9230   
                                                                             unfreeze       0.9224   
                      DroPTC-all-mpnet-base-v2-sentence ce      balanced     freeze         0.9265   
                                                                             unfreeze       0.9310   
                                                                inverse      freeze         0.9265   
                                                                             unfreeze       0.9320   
                                                                uniform      freeze         0.9135   
                                                                             unfreeze       0.9275   
                                                                uniformold   freeze         0.9135   
                                                                             unfreeze       0.9291   
                                                        focal   balanced     freeze         0.3752   
                                                                             unfreeze       0.9253   
                                                                inverse      freeze         0.9275   
                                                                             unfreeze       0.9335   
                                                                uniform      freeze         0.9247   
                                                                             unfreeze       0.9281   
                      all-MiniLM-L6-v2                  ce      uniform      freeze         0.7167   
                                                                             unfreeze       0.9214   
                      all-mpnet-base-v2                 ce      uniform      freeze         0.7139   
                                                                             unfreeze       0.9272   
                      bert-base-uncased                 ce      uniform      freeze         0.7355   
                                                                             unfreeze       0.9177   
          

### Message-level results

In [4]:
import json
import pandas as pd
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score

# scenario_name, embedding, seed = scenario_name.split('_')
# print(f'current scenario: {scenario}')
analysis_level = 'message'
source_dir = os.path.join('droptc', analysis_level)
recap_overall = pd.DataFrame()
recap_per_class = pd.DataFrame()
train_df = pd.read_excel(os.path.join('dataset', 'train_sentence.xlsx'))
true_df = pd.read_excel(os.path.join('dataset', 'test_sentence.xlsx'))
train_sentences = set(train_df['sentence'].unique())
embeddings = os.listdir(source_dir)
for embedding in embeddings:
    embedding_path = os.path.join(source_dir, embedding)
    freezes = os.listdir(embedding_path)
    for freeze in freezes:
        freeze_path = os.path.join(embedding_path, freeze)
        seeds = os.listdir(freeze_path)
        for seed in seeds:
            seed_path = os.path.join(freeze_path, str(seed))
            all_files = os.listdir(seed_path)
            eval_report_path = os.path.join(seed_path, 'evaluation_report.json')
            scenario_path = os.path.join(seed_path, 'scenario_arguments.json')
            with open(eval_report_path) as eval_report_file:
                eval_report = json.load(eval_report_file)
                with open(scenario_path) as scenario_json_file:
                    scenario = json.load(scenario_json_file)
                    prediction_path = os.path.join(seed_path, 'prediction.xlsx')
                    prediction_df = pd.read_excel(prediction_path)
                    # Identify non-leaked samples
                    # non_leaked_mask = ~prediction_df['sentence'].isin(train_sentences)

                    # # Filter prediction and ground truth
                    # filtered_pred_df = prediction_df[non_leaked_mask]
                    # filtered_true_df = true_df[non_leaked_mask]

                    # # Compute evaluation
                    # # Extract labels
                    # y_true = filtered_true_df['label'].values
                    # y_pred = filtered_pred_df['pred'].values
                    # unique_acc = accuracy_score(y_true, y_pred)
                    accuracy = float(round(eval_report['accuracy'], 3))
                    # print(f'accuracy: {accuracy} -> {unique_acc}')
                    # unique_f1 = f1_score(y_true, y_pred, average='weighted')
                    f1score = float(round(eval_report['weighted_avg']['f1-score'], 3))
                    # print(f'f1_score: {f1score} -> {unique_f1}')
                    precision = float(round(eval_report['weighted_avg']['precision'], 3))
                    recall = float(round(eval_report['weighted_avg']['recall'], 3))
                    best_epoch = int(scenario['best_epoch'])
                    columns = ['dataset', 'embedding', 'freeze_param', 'seed', 'best_epoch', 'accuracy', 'precision', 'recall', 'f1_score']
                    row_list = [analysis_level, embedding, freeze, seed, best_epoch, accuracy, precision, recall, f1score]
                    scenario_df = pd.DataFrame([row_list], index=None, columns=columns)
                    recap_overall = pd.concat([recap_overall, scenario_df], ignore_index=True)
if not os.path.exists(os.path.join('recap', analysis_level)):
    os.makedirs(os.path.join('recap', analysis_level))
recap_overall.to_excel(os.path.join(os.path.join('recap', analysis_level), f'recap_overall_{analysis_level}.xlsx'), index=False, float_format='%.5f')

In [5]:
grouper = ['dataset', 'embedding', 'freeze_param']
selected_columns = ['accuracy', 'precision', 'recall', 'f1_score']

multitask_agg = recap_overall.groupby(grouper)[selected_columns].agg(['mean', 'std']).sort_values(('f1_score', 'mean'), ascending=True)
multitask_agg.to_excel(os.path.join('recap', f'recap_overall_{analysis_level}_accuracy.xlsx'), float_format='%.3f')
multitask_agg

accuracy           precision            \
                                           mean       std      mean       std   
dataset embedding         freeze_param                                          
message modern-bert       freeze         0.4693  0.010822    0.7166  0.025079   
        bert-base-uncased freeze         0.4642  0.015245    0.7230  0.010349   
        neo-bert          freeze         0.6111  0.017078    0.8772  0.008917   
        drone-sbert       freeze         0.7557  0.006075    0.9110  0.003496   
        modern-bert       unfreeze       0.7425  0.023755    0.9155  0.011881   
        neo-bert          unfreeze       0.7812  0.022185    0.9240  0.017185   
        bert-base-uncased unfreeze       0.8017  0.024244    0.9212  0.016752   
        drone-sbert       unfreeze       0.8565  0.011227    0.9482  0.007983   

                                        recall           f1_score            
                                          mean       std     mean       std  
dataset embedding         freeze_param                                       
message modern-bert       freeze        0.6327  0.008407   0.6391  0.010300  
        bert-base-uncased freeze        0.6472  0.009818   0.6468  0.009449  
        neo-bert          freeze        0.7742  0.017178   0.7947  0.013630  
        drone-sbert       freeze        0.8603  0.004029   0.8711  0.003281  
        modern-bert       unfreeze      0.8602  0.014891   0.8762  0.013340  
        neo-bert          unfreeze      0.8859  0.016543   0.8954  0.014339  
        bert-base-uncased unfreeze      0.9058  0.014891   0.9076  0.014886  
        drone-sbert       unfreeze      0.9270  0.005831   0.9326  0.004427